In [5]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import h5py
import copy

import amici
from petab.C import *
import petab
import petab.C
import pypesto
import pypesto.petab
from pypesto.optimize import minimize
from pypesto.startpoint import uniform
from pypesto.engine import MultiProcessEngine, MultiThreadEngine
from pypesto.optimize.optimizer import FidesOptimizer
import pypesto.optimize as optimize

from pypesto.visualize import waterfall
from pypesto.visualize import parameters
from pypesto.visualize.model_fit import visualize_optimized_model_fit
from pypesto.visualize import profiles

import pypesto.profile as profile
from pypesto.optimize import ScipyOptimizer
from pypesto.profile.options import ProfileOptions

from pypesto.visualize.model_fit import visualize_optimized_model_fit, _get_simulation_rdatas

In [6]:
def hex_to_rgba_gradient(color1, color2, n):
    '''
    Create a gradient in rgba between two hex colors
    '''
    # Convert to rgba
    c1 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color1))
    c2 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color2))

    return [[(c1[i]*(n-j-1) + c2[i]*j)/(n-1) for i in range(4)] for j in range(n)]

# find the index for cut off based on Chi square distribution CI 95%
def find_cut_off_index(result, ci = 0.95):
    '''
    Find the cut off index for the data based on the Chi square distribution
    '''

    # calculate the chi square distribution
    cut_off_value = scipy.stats.chi2.ppf(ci, 1)

    # find the index
    best_fval = result.optimize_result.list[0].fval

    for i in range(len(result.optimize_result.list)):
        if result.optimize_result.list[i].fval > best_fval + cut_off_value:
            break
    
    return i - 1

def find_cut_off_x_trace(result, ci = 0.95, flatten = True):

    cut_off_value = scipy.stats.chi2.ppf(ci, 1)
    best_fval = result.optimize_result.list[0].fval

    # store the optimized x trace that are below the cut off value
    x_trace_within_cut_off = []
    if flatten:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off.append(x_trace[j])
    else:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            x_trace_within_cut_off_i = []
            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off_i.append(x_trace[j])
            x_trace_within_cut_off.append(x_trace_within_cut_off_i)

    return x_trace_within_cut_off

In [7]:
# Plot setting
plt.rcParams['font.size'] = 30

dpi = 100
wid = int(2560/dpi)
hei = int(1600/dpi)

In [8]:
# number of optimization runs
n_runs = 5000

In [9]:
np.random.seed(500)

In [10]:
# Define the folder where you want to save the figures
folder_path = "/Users/yuhongliu/Documents/OV/figures/ifac_first_model_corrected/n"+str(n_runs)+"_v1/"

# If the folder does not exist, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# optimization
hierarchical = False

petab_yaml = 'petab_files/init_model_noscaling_newnoise_scaled.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=hierarchical,
        model_name=f"INIT_Model",
    ).create_problem(force_compile=True)

problem.objective.amici_model.setAllStatesNonNegative()

# some model properties
print("Model parameters:", list(problem.objective.amici_model.getParameterIds()), "\n")
print("Model const parameters:", list(problem.objective.amici_model.getFixedParameterIds()), "\n")
print("Model outputs:   ", list(problem.objective.amici_model.getObservableIds()), "\n")
print("Model states:    ", list(problem.objective.amici_model.getStateIds()), "\n")

Compiling amici model to folder /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/first_model_assump_correction/amici_models/0.27.0/INIT_Model.
2024-11-11 13:46:00.486 - amici.petab.sbml_import - INFO - Importing model ...
2024-11-11 13:46:00.487 - amici.petab.sbml_import - INFO - Validating PEtab problem ...
2024-11-11 13:46:00.500 - amici.petab.sbml_import - INFO - Model name is 'INIT_Model'.
Writing model code to '/Users/yuhongliu/Documents/OV/models_refit_for_IFAC/first_model_assump_correction/amici_models/0.27.0/INIT_Model'.
2024-11-11 13:46:00.501 - amici.petab.sbml_import - INFO - Species: 0
2024-11-11 13:46:00.502 - amici.petab.sbml_import - INFO - Global parameters: 10
2024-11-11 13:46:00.502 - amici.petab.sbml_import - INFO - Reactions: 0
2024-11-11 13:46:00.509 - amici.petab.sbml_import - INFO - Observables: 1
2024-11-11 13:46:00.509 - amici.petab.sbml_import - INFO - Sigmas: 1
2024-11-11 13:46:00.510 - amici.petab.sbml_import - DEBUG - Adding output parameters to model: [

running build_ext
running AmiciBuildCMakeExtension
------------------------------ model_ext ------------------------------

==> Configuring:
$ cmake -S /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/first_model_assump_correction/amici_models/0.27.0/INIT_Model -B /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/first_model_assump_correction/amici_models/0.27.0/INIT_Model/build_model_ext -G Ninja -DCMAKE_MAKE_PROGRAM=/Users/yuhongliu/Documents/venv_ov/bin/ninja -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX:PATH=/Users/yuhongliu/Documents/OV/models_refit_for_IFAC/first_model_assump_correction/amici_models/0.27.0/INIT_Model/INIT_Model -DCMAKE_VERBOSE_MAKEFILE=ON -DCMAKE_PREFIX_PATH=/Users/yuhongliu/Documents/venv_ov/lib/python3.12/site-packages/amici -DAMICI_PYTHON_BUILD_EXT_ONLY=ON -DPython3_EXECUTABLE=/Users/yuhongliu/Documents/venv_ov/bin/python3

==> Building:
$ cmake --build /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/first_model_assump_correction/amici_models/0.27

2024-11-11 13:46:11.639 - amici.petab.sbml_import - INFO - Finished Importing PEtab model            (1.12E+01s)


Model parameters: ['rho', 'kappa', 'psi', 'alpha', 'beta', 'delta', 'sigma_add', 'sigma_mul'] 

Model const parameters: ['virus_injection'] 

Model outputs:    ['tumor_num'] 

Model states:     ['C_u', 'C_i', 'V'] 



# Gradient Check

In [11]:
startpoints = problem.get_startpoints(n_starts=2)

In [9]:
problem.objective.check_grad(
    x = startpoints[0],
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,-5.982198e+04,-59717.689040,-59921.618215,-59819.653628,203.929175,2.329891,3.894858e-05
kappa,-1.129325e+04,-11163.159060,-11412.188320,-11287.673690,249.029260,5.576622,4.940453e-04
psi,3.487144e+00,114.079204,-115.541584,-0.731190,229.620788,4.218335,5.769213e+00
beta,-5.776472e-07,121.155850,-120.000958,0.577446,241.156808,0.577447,9.999837e-01
alpha,3.550258e+00,119.948162,-116.494817,1.726672,236.442978,1.823586,1.056121e+00
delta,-3.311215e+00,121.073831,-120.728257,0.172787,241.802088,3.484002,2.016241e+01
sigma_add,-5.762296e+04,-57501.735436,-57744.192992,-57622.964214,242.457556,0.003531,6.127140e-08
sigma_mul,-2.791734e+04,-27796.800504,-28037.892094,-27917.346299,241.091590,0.002037,7.294871e-08


In [12]:
problem.objective.check_grad(
    x = problem.get_reduced_vector(startpoints[0]),
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,-5.982198e+04,-59717.689040,-59921.618215,-59819.653628,203.929175,2.329891,3.894858e-05
kappa,-1.129325e+04,-11163.159060,-11412.188320,-11287.673690,249.029260,5.576622,4.940453e-04
psi,3.487144e+00,114.079204,-115.541584,-0.731190,229.620788,4.218335,5.769213e+00
beta,-5.776472e-07,121.155850,-120.000958,0.577446,241.156808,0.577447,9.999837e-01
alpha,3.550258e+00,119.948162,-116.494817,1.726672,236.442978,1.823586,1.056121e+00
delta,-3.311215e+00,121.073831,-120.728257,0.172787,241.802088,3.484002,2.016241e+01
sigma_add,-5.762296e+04,-57501.735436,-57744.192992,-57622.964214,242.457556,0.003531,6.127140e-08
sigma_mul,-2.791734e+04,-27796.800504,-28037.892094,-27917.346299,241.091590,0.002037,7.294871e-08


In [13]:
gc = problem.objective.check_grad_multi_eps(
    x=startpoints[0],
    verbosity=0,
    label='rel_err',  # default
)
gc

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,-5.982198e+04,-59743.244776,-59900.691985,-59821.968380,157.447209,0.015138,2.530512e-07,1.000000e-03
kappa,-1.129325e+04,-11254.614206,-11332.218633,-11293.416419,77.604427,0.166108,1.470836e-05,1.000000e-03
psi,3.487144e+00,3.923703,3.108793,3.516248,0.814909,0.029104,8.048000e-03,1.000000e-01
beta,-5.776472e-07,0.011191,-0.011868,-0.000339,0.023060,0.000338,3.392192e-03,1.000000e-01
alpha,3.550258e+00,4.715786,2.360857,3.538321,2.354929,0.011937,3.372626e-03,1.000000e-03
delta,-3.311215e+00,-2.983684,-3.674916,-3.329300,0.691232,0.018085,5.600279e-03,1.000000e-01
sigma_add,-5.762296e+04,-45575.529512,-69670.398661,-57622.964086,24094.869150,0.003403,5.906171e-08,1.000000e-07
sigma_mul,-2.791734e+04,-27908.985257,-27925.699602,-27917.342430,16.714345,0.001833,6.564664e-08,1.000000e-03


In [14]:
def highlight_value_above_threshold(x, threshold=10):
    return ['color: darkorange' if xi > threshold else None for xi in x]

gc.style.apply(
    highlight_value_above_threshold, subset=["fd_err"],
).background_gradient(
    cmap=sns.light_palette("purple", as_cmap=True), subset=["abs_err"],
).background_gradient(
    cmap=sns.light_palette("red", as_cmap=True), subset=["rel_err"],
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,-59821.983518,-59743.244776,-59900.691985,-59821.968380,157.447209,0.015138,0.000000,0.001000
kappa,-11293.250312,-11254.614206,-11332.218633,-11293.416419,77.604427,0.166108,0.000015,0.001000
psi,3.487144,3.923703,3.108793,3.516248,0.814909,0.029104,0.008048,0.100000
beta,-0.000001,0.011191,-0.011868,-0.000339,0.023060,0.000338,0.003392,0.100000
alpha,3.550258,4.715786,2.360857,3.538321,2.354929,0.011937,0.003373,0.001000
delta,-3.311215,-2.983684,-3.674916,-3.329300,0.691232,0.018085,0.005600,0.100000
sigma_add,-57622.960683,-45575.529512,-69670.398661,-57622.964086,24094.869150,0.003403,0.000000,0.000000
sigma_mul,-27917.344262,-27908.985257,-27925.699602,-27917.342430,16.714345,0.001833,0.000000,0.001000
